In [ ]:
import re
from pathlib import Path

import hvplot.pandas
import hvplot.xarray
import pandas as pd
import pypsa
import seaborn
import xarray as xr

seaborn.set_style("whitegrid")

n = pypsa.Network()
n.import_from_netcdf(snakemake.input["network"])


def remove_numbers(s):
    """removes any numbers and surronding spaces from end of string,
    e.g. 'csp-tower TES 34' -> 'csp-tower TES'"""

    return re.sub("(\s*\d+\s*)$", "", s)


##+++ Plot SoCs (hourly, statistics)

df = n.stores_t["e"]
df = df.groupby(remove_numbers, axis="columns").sum()

fig_soc = df.hvplot(title="Store: SoC", ylabel="SoC [MWh, m3, t]", legend=False)
fig_soc_distribution = df.hvplot.violin(
    title="Store: SoC annual distribution", ylabel="SoC [MWh, m3, t]"
)

##+++ Plot SoCs (monthly means)
_df = df.groupby(pd.Grouper(freq="M")).mean()
_df.name = ""

_df.index.name = f"Month of {_df.index.year[0]}"  # Year as index name / xlabel in plot

_df.index = _df.index.month_name().str[
    :3
]  # 3-letter month names instead of last day of month

fig_soc_monthly_mean = _df.hvplot(
    title="Store: SoC monthly mean", ylabel="SoC [MWh, m3, t]"
).opts(legend_opts={"title": ""})

##+++ Plot capacities
df = n.stores.groupby(remove_numbers, axis="index")["e_nom_opt"].sum()
fig_capacities = df.hvplot.bar(
    title="Store: Capacities", ylabel="Max. capacity [MWh, m3, t]"
)

##+++ Plot cost
df = n.stores
df = df["e_nom_opt"] * df["capital_cost"]

fig_cost = (
    df.groupby(remove_numbers)
    .sum()
    .hvplot.bar(title="Store: Annualised cost", ylabel="Annual cost [EUR]")
)

##+++ Plot charging and discharging
df = n.stores_t["e"].groupby(remove_numbers, axis="columns").sum()

df = df.diff()

df_out = df[df < 0]

df_in = df[df > 0]

df_out = df_out.groupby(pd.Grouper(freq="D")).sum().fillna(0)
df_in = df_in.groupby(pd.Grouper(freq="D")).sum().fillna(0)

df_in.index = df_in.index.day_of_year
df_out.index = df_out.index.day_of_year

df_in.index.name = "Day"
df_out.index.name = "Day"

figs = None
for cn in df_in.columns:
    if n.stores.groupby(remove_numbers)["e_nom_opt"].sum()[cn] < 1:
        logger.info(
            f"Ignoring store {cn} due to negligible capacity in (Dis-) Charge plots."
        )
        continue

    fig = df_in.hvplot.area(
        x="Day",
        y=cn,
        title=f"Store: Daily flows ({cn})",
        label="charge",
        xlabel=f"Day of {df.index.year[0]}",
        ylabel="(Dis-) charge [MWh, m3, t]",
    ) * df_out.hvplot.area(x="Day", y=cn, label="discharge")

    fig = fig.opts(legend_position="right")
    figs = figs + fig if figs else fig

figs_flows_daily = figs

##+++ Combine figures
fig = (
    fig_soc
    + fig_soc_monthly_mean
    + fig_soc_distribution
    + fig_capacities
    + fig_cost
    + figs_flows_daily
).cols(2)

## +++ Save figures
for fp in snakemake.output["fig"]:
    hvplot.save(fig, fp)